In [ ]:
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as mp
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
consumerkey='133WQazwdixVCDHjMZjW09HwG'
consumersecret='6lsIjQQth5z5tU3dyDEP92BrCpy8xuckTQ1WEQEm6jGcVVzIaJ'
accessToken='1444301444916539394-7X9nRHwzLtKszZXhKSAUhALekClIU9'
accessTokenSecret='JRlQnHZplLr5o6HBBk8q5QWXtzY46FFGdCWcDJkyGcNFV'

In [ ]:
authenticate=tweepy.OAuthHandler(consumerkey,consumersecret)

authenticate.set_access_token(accessToken,accessTokenSecret)

api=tweepy.API(authenticate,wait_on_rate_limit=True)

In [ ]:
posts=api.search(q="SamsungIndia",lang="en",rrp=1000)
for tweet in posts:
  print(tweet.text)
for tweet in posts:
  print(tweet.user.name)

In [ ]:
df=pd.DataFrame( [tweet.text for tweet in posts],columns=['Tweets'])

df_add = pd.DataFrame([tweet.user.name for tweet in posts],columns=['Name'])
df = pd.concat([df,df_add], axis=1)
df.head(15)

In [ ]:
def cleanTxt(text):
  text=re.sub(r'@[A-Za-z0-9]+','',text)#Remove @mention
  text=re.sub(r'#','',text)#Removing the '#' symbol
  text=re.sub(r'RT[\s]+','',text)#Removing RT
  text=re.sub(r'https?:\/\/\S+','',text)#Remove the hyper link

  return text
df['Tweets']=df['Tweets'].apply(cleanTxt)

df

In [ ]:
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity


def getPolarity(text):
  return TextBlob(text).sentiment.polarity

df['Subjectivity']=df['Tweets'].apply(getSubjectivity)
df['Polarity']=df['Tweets'].apply(getPolarity)

df

In [ ]:
allWords=' '.join([twts for twts in df['Tweets']])
wordCloud=WordCloud(width=500,height=300,random_state=21,max_font_size=119).generate(allWords)

plt.imshow(wordCloud,interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
def getAnalysis(score):
  if score<0:
    return 'Negative'
  elif score==0:
    return 'Neutral'
  else:
    return 'Positive'

df['Analysis']=df['Polarity'].apply(getAnalysis)


df

In [ ]:
j=1
sortedDF=df.sort_values(by=['Polarity'])
for i in range(0,sortedDF.shape[0]):
  if (sortedDF['Analysis'][i]=='Positive'):
      print(str(j)+') '+sortedDF['Tweets'][i])
      print()
      j=j+1

In [ ]:
j=1
sortedDF=df.sort_values(by=['Polarity'],ascending='False')
for i in range(0,sortedDF.shape[0]):
  if (sortedDF['Analysis'][i]=='Negative'):
      print(str(j)+') '+sortedDF['Tweets'][i])
      print(str(j)+')'+sortedDF['Name'][i])
      print()
      j=j+1

In [ ]:
plt.figure(figsize=(8,6))
for i in range(0,df.shape[0]):
  plt.scatter(df['Polarity'][i],df['Subjectivity'][i],color='Blue')

plt.title('Sentiment Analysis')
plt.xlabel('Polarity')
plt.ylabel('Subjectivity')
plt.show()

In [ ]:
ptweets=df[df.Analysis=='Positive']
ptweets=ptweets['Tweets']

round((ptweets.shape[0]/df.shape[0] )*100,1)

In [ ]:
ntweets=df[df.Analysis=='Negative']
ntweets=ntweets['Tweets']

round((ntweets.shape[0]/df.shape[0] )*100,1)

In [ ]:
df['Analysis'].value_counts()

plt.title('Sentiment Alalysis')
plt.xlabel('Sentiment')
plt.ylabel('Counts')
df['Analysis'].value_counts().plot(kind='bar')
plt.show()